In [60]:
import gc
import os
import pandas as pd
import seaborn as sns

import matplotlib.pyplot as plt

# LOAD LIBRARY
import gc
import os
import random
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
plt.show()

import seaborn as sns
sns.set(style="whitegrid")

import warnings
warnings.filterwarnings('ignore')

# plotly
#pip install chart_studio
import plotly.express as px
import chart_studio.plotly as py
import plotly.graph_objs as go
from plotly.offline import iplot

import cufflinks as cf
cf.go_offline()
cf.set_config_file(world_readable=True, theme='polar')

import lightgbm as lgb

from time import time
from tqdm import tqdm_notebook
from sklearn import preprocessing
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score
warnings.simplefilter('ignore')
sns.set()
%matplotlib inline

In [79]:
train = pd.read_csv('./kaggle/train.csv')

In [80]:
# a = pd.read_csv('./kaggle/competition/sample_submission.csv')
# a

In [81]:
# train.info()

num_of_null = train.isnull().sum()
percent = (train.isnull().sum() / train.isnull().count() * 100)
pd.concat([num_of_null, percent], axis=1, keys=['# of null', 'Percent']).sort_values(by='Percent', ascending=False)



,# of null,Percent
id,0,0.0
age,0,0.0
workclass,0,0.0
fnlwgt,0,0.0
education,0,0.0
education_num,0,0.0
marital_status,0,0.0
occupation,0,0.0
relationship,0,0.0
race,0,0.0


In [64]:
train.columns

Index(['id', 'age', 'workclass', 'fnlwgt', 'education', 'education_num',
       'marital_status', 'occupation', 'relationship', 'race', 'sex',
       'capital_gain', 'capital_loss', 'hours_per_week', 'native_country',
       'income'],
      dtype='object')

In [65]:
# train = train.replace('?', np.nan)
to_encode = ['id', 'age', 'workclass', 'fnlwgt', 'education', 'education_num',
       'marital_status', 'occupation', 'relationship', 'race', 'sex',
       'capital_gain', 'capital_loss', 'hours_per_week', 'native_country',
       ]

encoded = pd.get_dummies(train[to_encode])
train.drop(columns=to_encode, axis=1, inplace=True)
train = pd.concat([train, encoded], axis=1)
train

,income,id,age,fnlwgt,education_num,capital_gain,capital_loss,hours_per_week,workclass_?,workclass_Federal-gov,...,native_country_Portugal,native_country_Puerto-Rico,native_country_Scotland,native_country_South,native_country_Taiwan,native_country_Thailand,native_country_Trinadad&Tobago,native_country_United-States,native_country_Vietnam,native_country_Yugoslavia
0,>50K,0,40,168538,9,0,0,60,0,0,...,0,0,0,0,0,0,0,1,0,0
1,<=50K,1,17,101626,5,0,0,20,0,0,...,0,0,0,0,0,0,0,1,0,0
2,<=50K,2,18,353358,10,0,0,16,0,0,...,0,0,0,0,0,0,0,1,0,0
3,<=50K,3,21,151158,10,0,0,25,0,0,...,0,0,0,0,0,0,0,1,0,0
4,<=50K,4,24,122234,10,0,0,20,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26044,<=50K,26044,57,250201,7,0,0,52,0,0,...,0,0,0,0,0,0,0,1,0,0
26045,<=50K,26045,23,238092,13,0,0,40,0,0,...,0,0,0,0,0,0,0,1,0,0
26046,<=50K,26046,78,165694,14,0,0,15,1,0,...,0,0,0,0,0,0,0,1,0,0
26047,<=50K,26047,26,151626,9,0,0,40,0,0,...,0,0,0,0,0,0,0,1,0,0


,# of null,Percent
income,0,0.0
native_country_Cambodia,0,0.0
native_country_Greece,0,0.0
native_country_Germany,0,0.0
native_country_France,0,0.0
...,...,...
marital_status_Divorced,0,0.0
education_Some-college,0,0.0
education_Prof-school,0,0.0
education_Preschool,0,0.0


In [66]:
test = pd.read_csv('./kaggle/test.csv')

In [77]:
# test = test.replace('?', np.nan)

num_of_null = test.isnull().sum()
percent = (test.isnull().sum() / test.isnull().count() * 100)
pd.concat([num_of_null, percent], axis=1, keys=['# of null', 'Percent']).sort_values(by='Percent', ascending=False)



,# of null,Percent
id,0,0.0
native_country_Canada,0,0.0
native_country_Guatemala,0,0.0
native_country_Greece,0,0.0
native_country_Germany,0,0.0
...,...,...
marital_status_Married-AF-spouse,0,0.0
marital_status_Divorced,0,0.0
education_Some-college,0,0.0
education_Prof-school,0,0.0


In [68]:
to_encode = ['id', 'age', 'workclass', 'fnlwgt', 'education', 'education_num',
       'marital_status', 'occupation', 'relationship', 'race', 'sex',
       'capital_gain', 'capital_loss', 'hours_per_week', 'native_country',
       ]

encoded = pd.get_dummies(test[to_encode])
test.drop(columns=to_encode, axis=1, inplace=True)
test = pd.concat([test, encoded], axis=1)
test

,id,age,fnlwgt,education_num,capital_gain,capital_loss,hours_per_week,workclass_?,workclass_Federal-gov,workclass_Local-gov,...,native_country_Portugal,native_country_Puerto-Rico,native_country_Scotland,native_country_South,native_country_Taiwan,native_country_Thailand,native_country_Trinadad&Tobago,native_country_United-States,native_country_Vietnam,native_country_Yugoslavia
0,0,28,67661,10,0,0,40,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,1,40,37869,9,0,0,50,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,2,20,109952,10,0,0,25,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,3,40,114537,11,0,0,50,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,4,37,51264,16,0,0,99,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6507,6507,35,61343,13,0,0,40,0,0,0,...,0,0,0,0,0,0,0,1,0,0
6508,6508,41,32185,13,0,0,40,0,0,0,...,0,0,0,0,0,0,0,1,0,0
6509,6509,39,409189,3,0,0,40,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6510,6510,35,180342,9,0,0,40,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [69]:
from lightgbm import LGBMClassifier
from sklearn.model_selection import train_test_split 




from sklearn.datasets import load_wine, load_breast_cancer, load_digits
from sklearn.model_selection import train_test_split

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [99]:
# y = train.income
# X = train.drop(columns=['income'])

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


data = train
x = data.iloc[:, :-1]
y = data.iloc[:, -1]
train_X,test_X,train_Y,test_Y=train_test_split(x,y,test_size=0.1, stratify=y)


In [108]:
train.iloc[-1]

id                        26048
age                          20
workclass                     ?
fnlwgt                    99891
education          Some-college
education_num                10
marital_status    Never-married
occupation                    ?
relationship          Own-child
race                      White
sex                      Female
capital_gain                  0
capital_loss                  0
hours_per_week               30
native_country    United-States
income                    <=50K
Name: 26048, dtype: object

In [112]:
lgbm = LGBMClassifier()
lgbm.fit(train_X, train_Y)
lgb_pred=lgbm.predict(test.drop(columns=['native_country_Holand-Netherlands']))

ValueError: DataFrame.dtypes for data must be int, float or bool.
Did not expect the data types in the following fields: workclass, education, marital_status, occupation, relationship, race, sex, native_country

In [104]:
print(classification_report(y_test, lgbm.predict(X_test)))

AttributeError: 'NoneType' object has no attribute 'predict'

In [105]:
sample_submission = pd.read_csv('kaggle/sample_submission.csv')
sample_submission['prediction'] = lgb_pred
sample_submission.to_csv('submission.csv', index=False)

ValueError: Length of values (5210) does not match length of index (6512)

In [106]:
sample_submission.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6512 entries, 0 to 6511
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   id          6512 non-null   int64
 1   prediction  6512 non-null   int64
dtypes: int64(2)
memory usage: 101.9 KB


In [96]:
sample_submission

,id,prediction
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0
...,...,...
6507,6507,0
6508,6508,0
6509,6509,0
6510,6510,0


In [83]:
len(train.columns) 

16

In [87]:
len(test.value())

AttributeError: 'DataFrame' object has no attribute 'value'